In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 957.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip your_path_to_the_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "nmdr/llama-2-7b-chat-platypus-1k-Baymax10"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access pub

tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

360

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1324

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  LLaMA-2 was trained on approximately 2 trillion tokens.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  LLaMA-2 was trained on approximately 2 trillion tokens.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip your_path_to_the_zip_file
!unzip -q Questionscsv.zip -d questions

###Mamba Questions

In [ ]:
# Mamba Questions

questions_file_path = '/content/questions/MambaQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Mamba achieves computational efficiency
through several strategies:

Firstly, it operates under standard GPU configurations, avoiding the need for specialized hardware adaptations. By utilizing generic GPU capabilities,
Mamba can handle various tasks without requiring customized hardware components.

Secondly, Mamba simplifies existing neural network architectures by combining RNN-like and CNN-like layers. This integration reduces the complexity
associated with traditional attention mechanisms while maintaining the effectiveness of MLP blocks in natural language processing tasks.

Lastly, Mamba introduces selective state space models, which enable dynamic parameter adjustments based on output. This reverse feedback mechanism
allows Mamba to optimize real-time processing capabilities and adjust computational focus accordingly, providing a more efficient approach to sequence
modeling compared to traditional models.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Mamba introduces two key innovations to manage long
sequence data processing:

1. Selective State Space Models (SSMs): Mamba employs mechanisms similar to Recurrent Neural Networks (RNNs) and Convolutional Neural Networks (CNNs)
to provide a more scalable approach to sequence modeling. Unlike traditional SSMs, which struggle with longer sequences, Mamba's selective SSMs
dynamically adjust model parameters based on the output, optimizing real-time processing capabilities and adjusting computational focus dynamically.
2. Reverse Feedback Mechanism: Mamba incorporates a reverse feedback mechanism that allows it to optimize model parameters based on the output. This
mechanism enables Mamba to adaptively adjust its parameters during training, leading to improved performance on long sequences.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Based on the given passage,
there are several areas where Mamba's performance evaluation suggests potential directions for future research and development:

1. Longer sequences and more complex datasets: While preliminary evaluations indicate that Mamba performs competently on shorter sequences, its
efficacy on longer sequences and more intricate datasets remains unexplored. Research could focus on expanding these evaluations to better understand
the model's capabilities.
2. Broader hardware optimization: Although Mamba operates effectively under standard GPU configurations, investigating other hardware adaptations
could lead to even greater improvements in performance. Examining alternative hardware setups or developing customized solutions tailored specifically
to Mamba might enhance its accessibility and speed up processing times.
3. Expanding the scope of Mamba beyond video and short text snippets: While initial tests demonstrate promise in these domains, exploring how Mamba
fare

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Dynamic parameter adjustment plays a
crucial role in Mamba's selective state space models. The reverse feedback mechanism allows model parameters to adapt based on the output, optimizing
real-time processing capabilities and adjusting computational focus dynamically.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Variance-based.pdf
/content/papers/Mamba.pdf
/content/papers/Variance-based.pdf


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Mamba addresses the
inefficiencies of transformers in moderate to long sequence processing by leveraging structured state space models (ssms) traditionally applied in
image processing. It simplifies existing neural network architectures by integrating rnn-like and cnn-like layers, thereby reducing complexity and
dependency on traditional attention mechanisms.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
  The design of Mamba has
significant implications for its applicability across different data modalities. Firstly, the use of structured state space models (SSMs) allows Mamba
to handle both sequential and non-sequential data types, such as images and videos, through the integration of mechanisms similar to recurrent neural
networks (RNNs) and convolutional neural networks (CNNs). This versatility enables Mamba to tackle various tasks beyond just text processing.

Secondly, the introductio

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: How does Mamba achieve computational efficiency without specialized hardware optimizations?
 response:   Mamba achieves computational efficiency
through several strategies:

Firstly, it operates under standard GPU configurations, avoiding the need for specialized hardware adaptations. By utilizing generic GPU capabilities,
Mamba can handle various tasks without requiring customized hardware components.

Secondly, Mamba simplifies existing neural network architectures by combining RNN-like and CNN-like layers. This integration reduces the complexity
associated with traditional attention mechanisms while maintaining the effectiveness of MLP blocks in natural language processing tasks.

Lastly, Mamba introduces selective state space models, which enable dynamic parameter adjustments based on output. This reverse feedback mechanism
allows Mamba to optimize real-time processing capabilities and adjust computational focus accordingly, providing a more efficient approach to sequence
model

In [ ]:
json_file_name = 'MambaQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'MambaQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Parametric Magnon Questions

In [ ]:
# Parametric Magnon Questions

questions_file_path = '/content/questions/ParametricMagnonQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Materials commonly utilized in
constructing quantum transducers in hybrid quantum systems include yttrium iron garnet (YIG) and nitrogen-vacancy (NV) defects in diamond. However,
there is also potential for other materials like silicon carbide (SiC) to be incorporated into these systems.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Magnon nonlinearities enhance quantum transduction by allowing for more
precise control over the interaction between microwaves and spin qubits. Unlike linear magnon dynamics, which rely solely on the collective spin
excitations in magnetic materials, nonlinear magnonics enables the manipulation of individual magnons through their intrinsic nonlinear properties.
This opens up new possibilities for tailoring the interaction between microwaves and spin qubits, leading to improved transduction efficiency and
tunability.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Parametric magnonics introduces an innovative
approach in quantum transducers by utilizing nonlinear magnon dynamics to enhance microwave transduction to spin qubits. Unlike traditional linear
magnon transducers, which rely solely on the dipole interaction between the spin centers and the linear magnons, the proposed hybrid system employs
parametric processes to selectively turn the spin-magnon coupling "on" and "off." This novel mechanism offers improved control over the coupling
strength, allowing for better protection of the spin centers against unwanted magnon noise.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Introducing nonlinear magnonics into quantum computing
systems can significantly impact their performance and capabilities. Nonlinear magnonics refers to the study of how magnon waves interact with each
other and with other particles in complex ways, leading to emergent phenomena such as self-organization and nonlinear effects. These properties can be
leveraged to improve the efficiency and accuracy of quantum computations. For example, nonlinear magnonics can enable more robust control over quantum
systems, allowing for faster optimization algorithms and improved error correction techniques. Additionally, it may lead to new types of quantum gates
and operations that take advantage of the unique properties of magnon waves. Overall, incorporating nonlinear magnonics into quantum computing systems
offers exciting possibilities for advancing the field and achieving breakthroughs in areas like cryptography, drug discovery, and material science.


Sources:
/content/papers/Parametric magno

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Future applications that could potentially
benefit from magnon nonlinearities in quantum systems include:

1. Quantum communication networks: Nonlinear magnonics could enable more efficient and secure transmission of quantum information over longer
distances.
2. Quantum computing architectures: Hybrid systems combining linear and nonlinear magnonics could lead to improved performance and scalability in
quantum computing devices.
3. Spintronic devices: Harnessing the power of magnon nonlinearities could revolutionize the development of spintronic devices such as
magnetoresistors, spin valves, and spin- torque oscillators.
4. Magnetic sensors: Advanced magnetic sensors leveraging nonlinear magnonics could provide higher sensitivity and resolution in various applications
like navigation, medical imaging, and material characterization.
5. Energy harvesting and storage: Nonlinear magnonics may offer new opportunities for energy conversion and storage technologies, including magnetic
flywh

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What materials are typically used to construct quantum transducers in hybrid quantum systems?
 response:   Materials commonly utilized in
constructing quantum transducers in hybrid quantum systems include yttrium iron garnet (YIG) and nitrogen-vacancy (NV) defects in diamond. However,
there is also potential for other materials like silicon carbide (SiC) to be incorporated into these systems.


Q1: How do magnon nonlinearities enhance quantum transduction?
 response:   Magnon nonlinearities enhance quantum transduction by allowing for more
precise control over the interaction between microwaves and spin qubits. Unlike linear magnon dynamics, which rely solely on the collective spin
excitations in magnetic materials, nonlinear magnonics enables the manipulation of individual magnons through their intrinsic nonlinear properties.
This opens up new possibilities for tailoring the interaction between microwaves and spin qubits, leading to improved transduction efficiency and
tunability.

In [ ]:
json_file_name = 'ParametricMagnonQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'ParametricMagnonQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Quantum Mechanics Questions

In [ ]:
# Quantum Mechanics Questions

questions_file_path = '/content/questions/QuantumMechanicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Quantum mechanics plays a crucial role in modern technology due
to its unique properties, such as superposition, entanglement, and wave-particle duality. These properties enable quantum systems to perform tasks
faster than their classical counterparts, leading to breakthroughs in various fields. Here are some examples of significant applications of quantum
mechanics in modern technology:

1. Supercomputing: Quantum computers use quantum bits (qubits) instead of classical bits to process information. Qubits can exist in multiple states
simultaneously, allowing quantum computers to solve complex problems exponentially faster than classical computers. For example, Shor's algorithm for
factorizing large numbers can efficiently crack encryption codes used in secure communication networks.
2. Cryptography: Quantum key distribution (QKD) utilizes entangled particles to create secure keys for encrypting messages. Since measuring an
entangled state collapses the state of both particles, eaves

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Quantum mechanics has greatly influenced several new fields of study,
including:

1. Quantum Information Theory: This field focuses on the application of quantum mechanical concepts to information processing and storage. It has led
to the development of quantum algorithms, quantum cryptography, and quantum teleportation.
2. Quantum Thermodynamics: This field studies the behavior of quantum systems under different conditions, such as temperature, pressure, and energy
exchange. It has helped researchers understand the fundamental limits of quantum efficiency and develop new cooling techniques.
3. Condensed Matter Physics: Quantum mechanics has played a crucial role in understanding the properties of materials at the atomic scale. By studying
the behavior of electrons in solids, researchers have discovered exotic phases like superconductivity, superfluidity, and graphene.
4. Statistical Mechanics: This field deals with the statistical interpretation of thermodynamic properties. Quantum 

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  There are several fundamental questions about quantum
mechanics that remain open, some of which include:

1. Interpretation of Quantum Mechanics: Despite significant efforts over the past century, there is still no consensus on the proper interpretation of
quantum mechanics. Different approaches such as Copenhagen interpretation, Many Worlds Interpretation, and Relational Quantum Mechanics continue to be
discussed and debated.
2. Quantum Measurement Problem: One of the most challenging problems in quantum mechanics is understanding what happens when a measurement is made on
a quantum system. The act of measurement appears to disturb the state of the system, leading to a loss of quantum coherence. Resolving this problem
requires developing a deeper understanding of the relationship between observation and reality.
3. Quantum Entanglement: Another longstanding mystery is the phenomenon of quantum entanglement, where two or more particles become correlated in such
a way that their prope

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Exploring quantum mechanics contributes significantly to
theoretical physics by providing insights into the behavior of matter and energy at the atomic and subatomic level. Quantum mechanics offers a
framework for understanding phenomena that cannot be explained solely through classical physics, such as superposition, entanglement, wave-particle
duality, and uncertainty principles. These concepts have led to numerous discoveries and technological breakthroughs, including transistors, lasers,
magnetic resonance imaging (MRI), and computer chips. Additionally, research in quantum mechanics continues to advance our understanding of the
fundamental laws governing the universe, potentially leading to new technologies and unlocking secrets about dark matter and dark energy.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Experimental advances in verifying quantum theory include the
ability to manipulate and measure individual photons, demonstration of quantum teleportation, and observation of quantum entanglement between two
particles separated by billions of kilometers. Additionally, experiments have shown evidence of quantum superposition, where a single particle exists
in multiple states simultaneously, and quantum eraser, where the act of measurement affects the outcome of a previous measurement. These developments
have helped establish quantum mechanics as a reliable framework for describing subatomic phenomena.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
  According to the text, future research should focus on creating
educational resources that differentiate between classical and quantum physical theories to avoid perpetuating misunderstandings in scientific and
technological advancement.

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the significance of quantum mechanics in modern technology?
 response:   Quantum mechanics plays a crucial role in modern technology due
to its unique properties, such as superposition, entanglement, and wave-particle duality. These properties enable quantum systems to perform tasks
faster than their classical counterparts, leading to breakthroughs in various fields. Here are some examples of significant applications of quantum
mechanics in modern technology:

1. Supercomputing: Quantum computers use quantum bits (qubits) instead of classical bits to process information. Qubits can exist in multiple states
simultaneously, allowing quantum computers to solve complex problems exponentially faster than classical computers. For example, Shor's algorithm for
factorizing large numbers can efficiently crack encryption codes used in secure communication networks.
2. Cryptography: Quantum key distribution (QKD) utilizes entangled particles to create secure keys for encrypting messag

In [ ]:
json_file_name = 'QuantumMechanicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'Quantum MechanicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Qubit Teleportation Questions

In [ ]:
# Qubit Teleportation Questions

questions_file_path = '/content/questions/QubitTeleportationQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the article, the main components used in
quantum networks for teleportation are:

* NV centres in diamonds as communication qubits
* Nearby 13C nuclear spins as memory qubits
* Optical connections between nodes
* Solid-state spin qubits

These components enable the establishment of remote entanglement between pairs of neighboring nodes and allow for the reliable transfer of quantum
information across lossy network connections.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Entanglement establishment between distant nodes in a
quantum network involves several steps. Firstly, entanglement swapping protocols are used to create entanglement between neighboring nodes. This
process involves applying a specific sequence of quantum gates to the qubits involved, which creates a shared entangled state between them. Next,
entanglement swapping is performed on the middle node, where the entangled state is transferred from one node to another through a quantum channel.
Finally, the entangled state is stored in a memory qubit located on the middle node.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Memory qubits play a crucial role in the
process of quantum teleportation in a network. They are used to store the information about the state of the qubit being teleported, which allows for
accurate reconstruction of the teleported state at the receiving end. During the teleportation process, the memory qubits are entangled with the
communication qubits, enabling the transfer of information between them. This entanglement is essential for maintaining the integrity of the
teleported state, as it ensures that the information stored in the memory qubits remains correlated with the original state of the qubit being
teleported. Additionally, the memory qubits provide a way to protect against errors caused by environmental noise, as they can be used to correct
errors that occur during the teleportation process. Overall, the presence of memory qubits significantly enhances the accuracy and reliability of
quantum teleportation in a network.


Sources:
/content/papers/Qubit teleportation.pdf

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Future applications of quantum teleportation in
quantum networks include multi-node protocols and applications utilizing the teleportation primitive.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Real-time feed-forward significantly impacts the
process of quantum teleportation by allowing for unconditional teleportation between non-connected nodes in the network. This is because it enables
the use of a deterministic Bell-state measurement (BSM) combined with real-time feed-forward, which enables unconditional teleportation. Without real-
time feed-forward, the BSM alone cannot achieve unconditional teleportation due to the limitations imposed by the need for high-quality entangled
resource states and efficient Bell-state measurements.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
  Technical
advancements required to achieve efficient quantum teleportation between non-neighboring nodes include:

* Improved remote entanglement generation techniques to increase the quality of shared entanglement between pairs of node

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What are the main components used in quantum networks for teleportation?
 response:   According to the article, the main components used in
quantum networks for teleportation are:

* NV centres in diamonds as communication qubits
* Nearby 13C nuclear spins as memory qubits
* Optical connections between nodes
* Solid-state spin qubits

These components enable the establishment of remote entanglement between pairs of neighboring nodes and allow for the reliable transfer of quantum
information across lossy network connections.


Q1: How is entanglement established between distant nodes in a quantum network?
 response:   Entanglement establishment between distant nodes in a
quantum network involves several steps. Firstly, entanglement swapping protocols are used to create entanglement between neighboring nodes. This
process involves applying a specific sequence of quantum gates to the qubits involved, which creates a shared entangled state between them. Next,
entanglement swapping is p

In [ ]:
json_file_name = 'QubitTeleportationQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QubitTeleportationQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Variance Based Questions

In [ ]:
# Variance Based Questions

questions_file_path = '/content/questions/VarianceQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The focus of variance-based sensitivity analysis in
quantum memory is to determine the sensitivity of a quantum memory implementation with fixed internal system parameters to changes in external
parameters while keeping the internal parameters ﬁxed.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  In the context of quantum physics, "Λ-type" refers to a specific type of quantum memory
called "resonant /Lambda1-type quantum memory." This type of memory utilizes the resonance effect in atoms to store and retrieve quantum information.
The term "Lambda" represents the characteristic energy level spacing of the atomic transition involved in the memory process.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Memory sensitivity analysis is significant because it helps identify areas
where small changes in input parameters could significantly impact overall system performance. By analyzing how different parameters affect the
system's behavior under various conditions, engineers can optimize their designs more effectively. Additionally, understanding the relationship
between parameters allows researchers to develop strategies for mitigating adverse effects caused by unforeseen variations in experimental conditions.
Overall, performing thorough memory sensitivity analyses enables scientists and engineers working with complex systems like quantum computers or
communication networks to better understand their capabilities and limitations, ultimately leading to improved design decisions and enhanced
reliability across multiple applications.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/cont

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Experimental techniques used to analyze memory sensitivity
include:

1. Variance-based sensitivity analysis: This method involves measuring the variance in memory efficiency due to changes in input parameters while
keeping internal system parameters fixed.
2. Two- and three-parameter sensitivity calculations: These methods involve calculating the fraction of observed variance in memory efficiency due to
correlations between parameters.
3. Single-photon detuning and chirp measurements: These experiments measure the effect of changing the optical field on the memory efficiency.
4. Upper bounds on memory fidelity: This approach calculates an upper bound on memory fidelity in the presence of experimental fluctuations and drift.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Gaussian control field relates to memory performance through
its impact on the optical control ﬁeld used in the memory interaction. The Gaussian control ﬁeld parameters, such as pulse area, delay, and duration,
are adjusted to optimize memory efficiency. When these parameters deviate from their optimal values due to drift or improper settings, it leads to
fluctuations in memory efﬁciency. These fluctuations can be quantified using the sensitivity analysis, which calculates the standard deviation in
memory efﬁciency caused by variations in each control ﬁeld parameter. This analysis helps identify regions where the memory protocols are most
sensitive to ﬂuctuations in the memory parameters. Additionally, understanding the relationship between the Gaussian control ﬁeld and memory
performance can inform strategies for optimizing memory systems.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the focus of variance-based sensitivity analysis in quantum memory?
 response:   The focus of variance-based sensitivity analysis in
quantum memory is to determine the sensitivity of a quantum memory implementation with fixed internal system parameters to changes in external
parameters while keeping the internal parameters ﬁxed.


Q1: What does Λ-type quantum memory refer to?
 response:   In the context of quantum physics, "Λ-type" refers to a specific type of quantum memory
called "resonant /Lambda1-type quantum memory." This type of memory utilizes the resonance effect in atoms to store and retrieve quantum information.
The term "Lambda" represents the characteristic energy level spacing of the atomic transition involved in the memory process.


Q2: Why are control field parameters crucial in Λ-type quantum memory systems?
 response:   Control field parameters are essential in Λ-type quantum
memory systems because they determine the interaction between the qubits and the 

In [ ]:
json_file_name = 'VarianceQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'VarianceQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)